# Libraries

In [10]:
import os
import math
import json

# Create FishNet tiles

In [15]:
def tile_to_lon(x, zoom):
    n = 2.0 ** zoom
    return x / n * 360.0 - 180.0

def tile_to_lat(y, zoom):
    n = 2.0 ** zoom
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
    return math.degrees(lat_rad)

def get_tile_bounding_box_coords(x, y, zoom=17):
    nw_lat = tile_to_lat(y, zoom)
    nw_lon = tile_to_lon(x, zoom)
    se_lat = tile_to_lat(y + 1, zoom)
    se_lon = tile_to_lon(x + 1, zoom)

    # GeoJSON polygon (counter-clockwise)
    return [
        [nw_lon, nw_lat],  # NW
        [se_lon, nw_lat],  # NE
        [se_lon, se_lat],  # SE
        [nw_lon, se_lat],  # SW
        [nw_lon, nw_lat]   # close loop
    ]

def create_geojson_from_folders(root_gsv_folder, selected_folder_list, zoom=17):
    features = []

    for subfolder in os.listdir(root_gsv_folder):
        try:
            x_str, y_str = subfolder.split("_")
            x = int(x_str)
            y = int(y_str)
        except ValueError:
            print(f"Skipping invalid folder name: {subfolder}")
            continue

        polygon = get_tile_bounding_box_coords(x, y, zoom)

        feature = {
            "type": "Feature",
            "properties": {
                "folder": subfolder
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [polygon]
            }
        }
        features.append(feature)

    geojson = {
        "type": "FeatureCollection",
        "features": features
    }

    return geojson

root_gsv_folder = "D:\\Phoenix\\gsv"
selected_folder_list = ["24776_52606", "24777_52606", "24776_52607"]  # Example list
zoom = 17

geojson_data = create_geojson_from_folders(root_gsv_folder, selected_folder_list, zoom)

output_path = "tile_grids.geojson"
with open(output_path, "w") as f:
    json.dump(geojson_data, f, indent=2)

Skipping invalid folder name: JSON
